In [3]:
# !pip install paramiko
# !pip install scp

In [1]:
import os
import boto3
import subprocess
import numpy as np
import time
import datetime
import paramiko
import io
from scp import SCPClient, SCPException
import sys
from ast import literal_eval

In [2]:
np.set_printoptions(threshold=sys.maxsize)

In [3]:
INSTANCE_SIZE = 6

In [4]:
client = boto3.client('ec2', region_name='us-east-1')
# Create SQS client
sqs = boto3.resource('sqs')

In [192]:
def get_default_security_group(client, key_name):
    #extract key_name attribute from the security groups returned
    response = [group[key_name] for group in client.describe_security_groups()['SecurityGroups'] if group['GroupName'] == 'default']

    return response

def get_key_pairs(client, removeExisting=False):
    if removeExisting:
        client.delete_key_pair(KeyName='airscholar-key')

    keypairs = client.describe_key_pairs()['KeyPairs']
    keypair = list(filter(lambda x: x['KeyName'] == 'airscholar-key', keypairs))

    if not keypair:
        keypair = client.create_key_pair(KeyName='airscholar-key')
        f = io.StringIO(keypair['KeyMaterial'])
        data = f.read()
        file = open('labsuser.pem', 'w')
        file.write(data)
        file.close()
    else:
        keypair = keypair[0]

    return keypair

def launch_new_instance(client, keypair, count):
    response = client.run_instances(
        ImageId='ami-05723c3b9cf4bf4ff',
        InstanceType='t2.micro',
        KeyName=keypair,
        MaxCount=count,
        MinCount=count,
        Monitoring={
            'Enabled': True
        },
        SecurityGroupIds= get_default_security_group(client, key_name='GroupId')
    )
    ec2_inst_ids = [res["InstanceId"] for res in response]
    waiter = client.get_waiter('instance_running')
    waiter.wait(InstanceIds=[ec2_inst_ids])
    return ec2_inst_ids

def prepare_instances(client, keypair, count):
    ec2 = boto3.resource('ec2')
    ec2_inst_ids = []

    deployed_count = 0
    for instance in ec2.instances.all():
        deployed_count += 1
        if instance.state['Name'] == 'running':
            ec2_inst_ids.append(instance.id)

    if deployed_count < count:
        ec2_inst_ids.append(launch_new_instance(client, keypair, (count - deployed_count)))

    if not ec2_inst_ids:
        ec2_inst_ids.append(launch_new_instance(client, keypair, count))

    return ec2, ec2_inst_ids

def configure_ssh():
    sshs = []
    for count in range(0, INSTANCE_SIZE):
        ssh = paramiko.SSHClient()
        ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
        sshs.append(ssh)
    return sshs

def ssh_connect_with_retry(ssh, ip_address, retries):
    if retries > 3:
        return False
    f = open('labsuser.pem', 'r')
    privkey = paramiko.RSAKey.from_private_key(f)
    # print(privkey)
    interval = 5
    try:
        retries += 1
        print('SSH into the instance: {}'.format(ip_address))
        ssh.connect(hostname=ip_address,
                    username='ec2-user', pkey=privkey)
        return True
    except Exception as e:
        print(e)
        time.sleep(interval)
        print('Retrying SSH connection to {}'.format(ip_address))
        ssh_connect_with_retry(ssh, ip_address, retries)

def ssh_disconnect(ssh):
        """Close ssh connection."""
        if ssh:
            ssh.close()

def get_public_address(ec2, instance_id):
    # ec2 = boto3.resource('ec2', region_name='us-east-1')
    instance = ec2.Instance(id=instance_id)
    instance.wait_until_running()
    current_instance = list(ec2.instances.filter(InstanceIds=[instance_id]))
    ip_address = current_instance[0].public_ip_address
    return ip_address

def get_queue(sqs, queue_name):
    # Get the queue. This returns an SQS.Queue instance
    # There is no queue, create a new SQS queue
    attributes = {
        'DelaySeconds': '0',
        'MessageRetentionPeriod': '86400',
        "ReceiveMessageWaitTimeSeconds": "0"
    }

    for idx in range(INSTANCE_SIZE):
        sqs.create_queue(
            QueueName=f"{queue_name}{idx}",
            Attributes=attributes
        )

        sqs.create_queue(
            QueueName=f'result-queue-{idx}',
            Attributes=attributes
        )

def send_message_to_queue(sqs, queue_name, message):
    queue = sqs.get_queue_by_name(QueueName=queue_name)

    # Send message to SQS queue
    response = queue.send_messages(
        Entries=message
    )
    return response

def install_required_packages(ssh):
    stdin, stdout, stderr = ssh.exec_command("sudo yum install pip -y && sudo pip install numpy boto3 AST")
    return stdout, stderr

def get_messages_from_queue(instance_size, queue, message_size=10):
    messages = []
    sqs = boto3.resource('sqs')
    queue = sqs.get_queue_by_name(QueueName=queue)
    #
    # client = boto3.client('sqs')
    # response = client.receive_message(
    #     QueueUrl = client.get_queue_url(QueueName=queue)['QueueUrl'],
    #     AttributeNames=['All'],
    #     MaxNumberOfMessages=10
    # )['Messages']
    # response.delete()

    for message in queue.receive_messages(MaxNumberOfMessages=message_size, MessageAttributeNames=['All'], WaitTimeSeconds=0):
        messages.append(message.body)
        message.delete()
    return messages

def split_row(array, nrows, ncols):
    """
    Return an array of shape (n, nrows, ncols) where
    n * nrows * ncols = arr.size

    If arr is a 2D array, the returned array should look like n subblocks with
    each subblock preserving the "physical" layout of arr.
    """
    h, w = array.shape
    assert h % nrows == 0, f"{h} rows is not evenly divisible by {nrows}"
    assert w % ncols == 0, f"{w} cols is not evenly divisible by {ncols}"
    return (array.reshape(h//nrows, nrows, -1, ncols)
               .swapaxes(1,2)
               .reshape(-1, nrows, ncols))

def split_col(array, nrows, ncols):
    """Split a matrix into sub-matrices."""
    r, h = array.shape
    return [np.vsplit(i, 5) for i in np.hsplit(arr1, r)]

def generate_array(nrows, ncols):
    arr = np.random.randint(10, size=(nrows, ncols))
    # # print('arr 1:\n', arr)
    # arr = split_row(arr, 1, split_size)
    arr1 = np.random.randint(20, size=(nrows, ncols))
    # print('arr 2:\n', arr1)
    # arr1 = split_col(arr1, 1, split_size)

    return arr, arr1

def upload_file_to_s3(file_name, bucket, object_name=None):
    """Upload a file to an S3 bucket

    :param file_name: File to upload
    :param bucket: Bucket to upload to
    :param object_name: S3 object name. If not specified then file_name is used
    :return: True if file was uploaded, else False
    """

    # If S3 object_name was not specified, use file_name
    if object_name is None:
        object_name = os.path.basename(file_name)

    # Upload the file
    s3_client = boto3.client('s3')
    try:
        response = s3_client.upload_file(file_name, bucket, object_name)
    except:
        # logging.error(e)
        return False
    return True

def bulk_upload(scp, filepaths: list[str], remote_path, host):
        """
        Upload multiple files to a remote directory.

        :param List[str] filepaths: List of local files to be uploaded.
        """
        try:
            scp.put(
                filepaths,
                remote_path=remote_path,
                recursive=True
            )
            print(f"Finished uploading {len(filepaths)} files to {remote_path} on {host}")
        except SCPException as e:
            print(f"SCPException during bulk upload: {e}")
        except Exception as e:
            print(f"Unexpected exception during bulk upload: {e}")

def configure_aws_access_for_ssh(ssh, ip_address):
    """
    This function extracts the AWS configuration you have locally and push to the server
    :param ssh:ssh object
    :return:
    """
    output = subprocess.getoutput("cat ~/.aws/credentials")
    ssh.exec_command(f'mkdir ~/.aws && touch ~/.aws/credentials')
    ssh.exec_command(f"echo '{output}' > ~/.aws/credentials")
    print(f'SSH AWS configuration done for {ip_address}')

def matrix_dot_product(matrix_a, matrix_b):
    start_time = datetime.datetime.now()
    result = []
    for i in range(len(matrix_a)):
        row = []
        for j in range(len(matrix_b[0])):
            sum = 0
            for k in range(len(matrix_b)):
                sum += matrix_a[i][k] * matrix_b[k][j]
            row.append(sum)
        result.append(row)
    print('Computation time', datetime.datetime.now() - start_time)

    return result

def matrix_add(matrix_1, matrix_2):
    start_time = datetime.datetime.now()
    result = []
    for idx_row in range(0, len(matrix_1)):
        row = matrix_1[idx_row]
        row1 = matrix_2[idx_row]
        cols = []
        for idx_col in range(0, len(row)):
            cols.append(row[idx_col] + row1[idx_col])
        result.append(cols)
    print('Computation time', datetime.datetime.now() - start_time)
    return result

In [6]:
QUEUE_NAME = 'queue'

In [7]:
get_queue(sqs, QUEUE_NAME)

In [46]:
sshs = configure_ssh() 
keypair = get_key_pairs(client, False)
ec2, instances = prepare_instances(client, keypair['KeyName'], INSTANCE_SIZE)
ip_addresses = [get_public_address(ec2, instance) for instance in instances]

for idx in range(0, len(sshs)):
    ssh = sshs[idx]
    ip_address = ip_addresses[idx]
    ssh_connect_with_retry(ssh, ip_address, 0)

SSH into the instance: 3.95.28.200
SSH into the instance: 107.20.122.207
SSH into the instance: 44.202.55.80
SSH into the instance: 54.172.123.86
SSH into the instance: 54.81.138.118
SSH into the instance: 54.86.228.162


In [9]:
# ssh_disconnect(ssh)
for idx in range(len(sshs)):
    ssh = sshs[idx]
    ip_address = ip_addresses[idx]
    stdout, stderr = install_required_packages(ssh)
    # print(stdout.read().decode('utf-8'))
    # print(stderr.read().decode('utf-8'))
    configure_aws_access_for_ssh(ssh, ip_address)

SSH AWS configuration done for 3.95.28.200
SSH AWS configuration done for 107.20.122.207
SSH AWS configuration done for 44.202.55.80
SSH AWS configuration done for 54.172.123.86
SSH AWS configuration done for 54.81.138.118
SSH AWS configuration done for 54.86.228.162


In [59]:
from files.file_helper import fetch_local_files
for idx in range(len(sshs)):
    ssh = sshs[idx]
    ip_address = ip_addresses[idx]
    scp = SCPClient(ssh.get_transport())
    bulk_upload(scp, fetch_local_files('./worker'), '~', ip_address)

Finished uploading 3 files to ~ on 3.95.28.200
Finished uploading 3 files to ~ on 107.20.122.207
Finished uploading 3 files to ~ on 44.202.55.80
Finished uploading 3 files to ~ on 54.172.123.86
Finished uploading 3 files to ~ on 54.81.138.118
Finished uploading 3 files to ~ on 54.86.228.162


## TODO: START THE WORKER ON ALL THE INSTANCES

In [61]:
# stdin, stdout, stderr = sshs[0].exec_command('cat worker.py')
# print(stdout.read().decode('utf-8'))

In [12]:
def start_worker(ssh, worker_id):
    stdin, stdout, stderr = ssh.exec_command(f'python ./worker.py {worker_id}')
    # result = stdout.read().decode('utf-8')
    # return result

In [62]:
# stdin, stdout, stderr = sshs[0].exec_command(f'python ./worker.py 1')
# print(stdout.read().decode('utf-8'))
# print(stderr.read().decode('utf-8'))

In [63]:
for idx in range(len(sshs)):
    ssh = sshs[idx]
    ip_address = ip_addresses[idx]
    start_worker(ssh, idx)
# sqs = boto3.resource('sqs')
# sqs
# response = send_message_to_queue(sqs, QUEUE_NAME, "Hi there! This is my queue message4!", 1, 0)
# response = send_message_to_queue(sqs, QUEUE_NAME, 'X', 1, 0)

### ADDITION

In [51]:
ARRAY_SIZE = 100

In [52]:
arr, arr1 = generate_array(ARRAY_SIZE,ARRAY_SIZE)

In [55]:
INT_ARRAY_SIZE = 10 #int(ARRAY_SIZE/INSTANCE_SIZE)
s_arr = split_row(arr, INT_ARRAY_SIZE, INT_ARRAY_SIZE)
s_arr1 = split_row(arr1, INT_ARRAY_SIZE, INT_ARRAY_SIZE)

In [56]:
print(s_arr[1])
print(s_arr1[1])

[[8 7 4 4 3 0 4 5 8 6]
 [5 5 9 0 4 9 0 7 2 1]
 [0 5 3 7 0 6 1 1 5 1]
 [1 5 4 7 4 8 3 6 6 0]
 [9 4 3 4 6 6 4 1 6 0]
 [5 4 2 2 7 5 5 2 7 1]
 [4 5 8 4 8 2 9 9 1 2]
 [6 3 3 3 2 3 0 1 1 9]
 [8 3 4 2 6 6 4 6 0 6]
 [9 5 1 3 9 4 8 4 4 2]]
[[11 11  9 15  4  7 11  6 10  4]
 [14  9 15  4  1 15 18 16  6  1]
 [17 16  3 18 17  4 19 16 19  7]
 [10  9  4  8 13 10  0  0 11  1]
 [ 5  5 16  2  2  6 16  4 16 12]
 [ 9  4 16  1  7  1  7  5 11  9]
 [16 16  9  8 17  6 16 18 15  2]
 [16  2  8  0  7  7  4  7 18  3]
 [ 6 16  3 14  9 14  2 16  5 18]
 [ 1  5  2  7 16 15 15 18 16  6]]


In [23]:
# iter = len(s_arr)/100

In [24]:

# b = str(b).replace('\n', ',').replace(' ', '')

In [25]:
# for y in range(0, 20)
# for x in range(0, iter):
# a = arr[0]
# b = arr[1]
# a = str(a).replace(' ', ',').replace('\n', '')
# b = str(b).replace('\n', ',').replace(' ', '')
    
    # stdin, stdout, stderr = ssh.exec_command(f'python dot_prod.py {a} {b}')
    # result = stdout.read().decode("utf-8").replace('\n', '')
    # literal_eval(result)[0][0]
    # print(stderr.read().decode("utf-8"))

In [200]:
def reformat_data(data):
    return str(data).replace('\n', '')
# a = s_arr[0]
# b = s_arr[1]
# a = str(a).replace(' ', ',').replace('\n', '')
# b = str(b).replace(' ', ',').replace('\n', '')
# ROW_LENGTH = int(len(s_arr)/INSTANCE_SIZE)
# messages = []
data = np.arange(0, len(s_arr))
for id, dt in enumerate(np.array_split(data, INSTANCE_SIZE)):
    # [print(f'queue{id}', {"Id": f"{idx+1}", "MessageBody": str((idx, (reformat_data(s_arr[idx]), reformat_data(s_arr1[idx])))) })
    #             for idx in range(min(dt), max(dt)+1)]
    [send_message_to_queue(sqs, f'queue{id}', [{"Id": f"{idx+1}", "MessageBody": str((idx, (reformat_data(s_arr[idx]), reformat_data(s_arr1[idx])))) }])
                for idx in range(min(dt), max(dt)+1)]
    
# for index in np.array_split(data, INSTANCE_SIZE):
#     # print(index)
#     messages.append([{"Id": f"{idx+1}", "MessageBody": str((idx, (reformat_data(s_arr[idx]), reformat_data(s_arr1[idx])))) }
#                 for idx in range(min(index), max(index)+1)])
    # if index <4:
    #     print(messages)
# messages1 = [{"Id": f"{idx}", "MessageBody": str((reformat_data(s_arr[idx]), reformat_data(s_arr1[idx]))) } for idx in range(5, 10)]
# messages2 = [{"Id": f"{idx}", "MessageBody": str((reformat_data(s_arr[idx]), reformat_data(s_arr1[idx]))) } for idx in range(10, 15)]
# messages3 = [{"Id": f"{idx}", "MessageBody": str((reformat_data(s_arr[idx]), reformat_data(s_arr1[idx]))) } for idx in range(15, 20)]
# messages4 = [{"Id": f"{idx}", "MessageBody": str((reformat_data(s_arr[idx]), reformat_data(s_arr1[idx]))) } for idx in range(20, 25)]

In [21]:
# messages

In [26]:

     
# [send_message_to_queue(sqs, 'queue0', messages[idx]) for idx in range(0, 4)]
# [send_message_to_queue(sqs, 'queue1', messages[idx]) for idx in range(4, 8)]
# [send_message_to_queue(sqs, 'queue2', messages[idx]) for idx in range(8, 12)]
# [send_message_to_queue(sqs, 'queue3', messages[idx]) for idx in range(12, 16)]
# [send_message_to_queue(sqs, 'queue4', messages[idx]) for idx in range(16, 20)]
# [send_message_to_queue(sqs, 'queue5', messages[idx]) for idx in range(20, 24)]

    # print(f'queue{int((idx * INSTANCE_SIZE) % INSTANCE_SIZE)}')
        # send_message_to_queue(sqs, f'queue{int(idx/2)}', messages[idx])

0 17
1 17
2 17
3 17
4 16
5 16


In [201]:
compute_res = []

In [203]:
# for a,b in enumerate(np.array_split(data, INSTANCE_SIZE)):
for i in range(17):
    res = get_messages_from_queue(INSTANCE_SIZE, f'result-queue-{0}', 1)
    # compute_res.append(res)
    [compute_res.append(msg) for msg in res]


In [204]:
compute_res

["(5, '[[457 350 577 574 620 501 317 573 456 266] [568 401 597 655 669 516 350 529 545 276] [431 409 504 483 519 400 409 502 394 237] [515 526 572 635 706 610 399 618 685 320] [298 361 413 344 476 510 370 403 379 282] [420 443 527 597 557 407 491 575 452 201] [403 347 470 510 588 374 281 405 442 182] [228 282 245 307 344 281 232 298 359 137] [259 304 265 319 373 388 320 288 305 186] [428 383 475 536 558 362 359 510 505 193]]')",
 "(1, '[[507 498 365 444 436 496 528 579 576 310] [504 351 419 309 332 271 490 414 566 280] [308 283 242 221 274 261 234 281 304 188] [491 391 409 287 379 344 381 398 528 296] [446 435 420 371 326 332 446 395 485 310] [400 397 395 307 311 327 429 413 463 317] [644 489 471 374 496 386 640 551 750 270] [257 253 229 264 302 300 371 340 400 177] [468 357 425 312 389 335 534 436 622 253] [515 458 499 353 375 392 562 478 604 307]]')",
 "(13, '[[535 655 515 517 474 506 586 440 600 364] [496 580 472 470 445 465 451 528 597 455] [278 426 236 266 222 218 201 261 336 282]

In [125]:
# a.append(compute_res)

In [126]:
a

[["(0, '[[451 378 376 320 455 349 358 291 449 424] [374 207 279 176 268 227 179 150 273 295] [400 651 434 451 687 447 307 225 502 384] [291 509 314 394 532 532 347 185 348 461] [369 522 484 465 708 498 256 246 491 424] [467 479 382 437 491 401 293 256 362 522] [324 409 312 391 629 431 258 288 400 519] [574 709 560 521 783 738 434 257 535 596] [513 583 451 443 707 420 331 314 604 593] [638 606 602 514 731 529 377 348 638 700]]')",
  "(15, '[[430 642 543 548 349 477 382 545 595 404] [273 361 327 243 230 344 161 311 382 176] [387 470 454 420 310 339 394 488 495 303] [388 479 511 381 261 535 320 523 484 302] [556 754 783 556 489 743 423 723 740 509] [444 447 422 348 274 375 305 487 436 310] [437 593 667 545 545 582 450 592 735 412] [559 638 626 515 473 647 392 645 646 463] [463 612 599 519 465 486 384 576 593 481] [613 629 612 510 517 659 376 625 700 449]]')",
  "(10, '[[503 334 473 322 506 461 263 175 292 445] [428 384 556 238 593 433 218 244 305 381] [545 302 398 331 551 401 367 130 334 

In [131]:
len(a)

13

In [129]:
# compute_res = [literal_eval(compute) for compute in compute_res]

In [66]:
len(compute_res)

6

In [67]:
compute_res

[(7,
  '[[391 440 471 258 320 403 205 493 317 427] [503 596 634 512 549 596 346 590 500 624] [152 178 211 139 154 138  87 218 147 186] [481 517 663 556 348 575 309 664 340 453] [382 352 378 439 239 366 142 515 394 417] [274 279 268 399 183 254  72 370 255 338] [429 466 463 318 365 372 215 435 306 369] [557 684 762 463 557 557 405 641 553 615] [472 463 535 470 313 514 228 599 292 407] [458 489 622 457 442 558 312 541 420 435]]'),
 (29,
  '[[393 478 406 232 400 444 376 357 364 274] [352 508 657 248 478 564 282 387 512 322] [446 370 372 324 381 345 269 420 575 352] [295 500 459 195 332 360 254 275 463 348] [361 393 320 373 412 440 315 371 447 381] [449 558 545 301 459 427 400 456 554 390] [385 405 318 332 391 342 354 419 566 433] [468 537 462 403 464 440 456 515 641 455] [406 314 348 357 375 393 277 450 472 310] [202 351 238 217 228 251 333 258 345 167]]'),
 (37,
  '[[601 434 459 545 406 583 501 573 409 691] [554 408 305 480 220 386 502 457 300 531] [662 561 499 705 437 588 709 705 354 80

In [204]:
# # sqs = boto3.resource('sqs')
# queue = sqs.(QueueName='airscholar-queue')
# for message in sqs.receive_message(
#             MaxNumberOfMessages=10):
#         # process message body
#         body = json.loads(message.body)
#         print(body)


In [143]:
# message = response['Messages']
# receipt_handle = message['ReceiptHandle']

# # Delete received message from queue
# sqs.delete_message(
#     QueueUrl=queue_url,
#     ReceiptHandle=receipt_handle
# )


In [196]:
# print(json.dumps(message, indent=4))

In [45]:
# # ec2_inst_id
# bucketName = 'airscholar-mlbd-bucket'

In [146]:
# s3 = boto3.client('s3')

In [ ]:
# response = s3.get_object(Bucket=bucketName,
#                          Key='data.json')
# print("Done, response body:")
# print(response['Body'].read())

In [ ]:
#create instance
#configure instance
#create matrix
#split matrix
#send matrix to the queue
#read matrix from the queue on the instance created
#compute matrix
#send result to base


In [58]:
!git add .
!git commit -am "Fixed result gathering from the queue"
!git push --set-upstream origin main

[main af0163d] Update infrastructure implementation. Fixed worker computation
 22 files changed, 6858 insertions(+), 670 deletions(-)
 create mode 100644 .idea/other.xml
 create mode 100755 .ipynb_checkpoints/Untitled-checkpoint.ipynb
 create mode 100755 .ipynb_checkpoints/boto3-aws-Copy1-checkpoint.ipynb
 create mode 100755 Untitled.ipynb
 delete mode 100644 __pycache__/dot_prod.cpython-39.pyc
 create mode 100755 boto3-aws-Copy1.ipynb
 create mode 100644 boto3-aws-copy.ipynb
 create mode 100644 config/aws-setup.py
 delete mode 100644 dot_prod.py
 rename files/{files.py => .ipynb_checkpoints/file_helper-checkpoint.py} (73%)
 mode change 100644 => 100755
 create mode 100644 files/__init__.py
 create mode 100644 files/__pycache__/__init__.cpython-39.pyc
 create mode 100644 files/__pycache__/file_helper.cpython-39.pyc
 delete mode 100644 files/__pycache__/files.cpython-39.pyc
 rename files/{.ipynb_checkpoints/files-checkpoint.py => file_helper.py} (73%)
Enumerating objects: 35, done.
Coun